# Document Search Pipeline

Ask anything related to documment and get answer based on the context from document.

In [1]:
! pip install vectorshift --upgrade


[notice] A new release of pip is available: 23.3.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Pipeline Overview

This pipeline takes two inputs, a document and a question. You will get answer from LLM based on context provided from the document.
![alt text](images/document_search/1-overview.png "pipeline overview")


In [21]:
import vectorshift as vs
from vectorshift.node import InputNode, URLLoaderNode, TextNode, SemanticSearchNode, OpenAILLMNode, OutputNode, ChatMemoryNode
from vectorshift.pipeline import Pipeline
from vectorshift.knowledge_base import *

In [22]:
vs_api_key = "YOUR_API_KEY_HERE"
vs.api_key = vs_api_key

## Input Nodes
Input seperated into two, document input and question. File loader is included in file loader, so you dont need to define it.
![alt text](images/document_search/2-inputs.png)

In [23]:
questions_input = InputNode(name="question",input_type="text")
document_input = InputNode(name="document_input",input_type="file", process_files=True)

In [24]:
search_node = SemanticSearchNode(
    query_input=[questions_input.output()],
    documents_input=[document_input.output()], 
    max_docs_per_query=4)

In [25]:
system_text = """You are a helpful assistant that answers User Question based on Context"""
system_text_node = TextNode(text=system_text)

In [26]:
llm = OpenAILLMNode(
    model="gpt-3.5-turbo",
    system_input=system_text_node.output(),
    prompt_input=questions_input.output(),
    text_inputs={"context":search_node.output()}
    )

In [27]:
output_node = OutputNode(name="output",output_type="text",input=llm.output())

## Deploy Pipeline

In [30]:
document_search_pipeline_nodes = [
    document_input, questions_input, search_node, llm, system_text_node, output_node
]

In [31]:
document_search_pipeline = Pipeline(
    name="Document Search with Vectorshift",
    description="Ask your document questions and get answers",
    nodes=document_search_pipeline_nodes
)

In [33]:
config = vectorshift.deploy.Config(
    api_key=vs_api_key,
)

config.save_new_pipeline(document_search_pipeline)

Successfully saved pipeline.


AttributeError: 'dict' object has no attribute 'json'

## Run The Pipeline

In [37]:
pipeline = Pipeline.fetch(pipeline_name='Document Search with Vectorshift')

response = pipeline.run(
    inputs = {"question": "What is Ekki's last name?", "document_input": "cv.pdf"},
    api_key= vs_api_key
)

print(response)

{}
